In [117]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import os
import cv2

In [118]:
train_path='../input/train/train'
test_path='../input/test/test'

In [119]:
train = pd.read_csv("../input/train.csv")
train.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [120]:
id=train['id'].values
l=train['has_cactus'].values

train=[]
X=[]
Y=[]
a=0

for i in tqdm(sorted(os.listdir(train_path))):
    path=os.path.join(train_path,i)
    i=cv2.imread(path,cv2.IMREAD_COLOR)
    X.append(i)
    train.append([np.array(i),l[a]])
    a=a+1

train=np.array(train)
Y=train[:,1]
train=train[:,0]
X=np.array(X)

X.shape

X=X/255
train=train/255

100%|██████████| 17500/17500 [00:16<00:00, 1051.67it/s]


In [121]:
test_viz=[]
X_test=[]

for i in tqdm(os.listdir(test_path)):
    id=i
    path=os.path.join(test_path,i)
    i=cv2.imread(path,cv2.IMREAD_COLOR)
    X_test.append(i)
    test_viz.append([np.array(i),id])

X_test=np.array(X_test)
X_test.shape
test_viz=np.array(test_viz)
id_test=test_viz[:,1]
test_viz=test_viz[:,0]
test_viz.shape

X_test=X_test/255
test_viz=test_viz/255

100%|██████████| 4000/4000 [00:04<00:00, 954.98it/s] 


### Creating Model

In [122]:
model = Sequential()
model.add(Conv2D(filters=128,kernel_size=2,padding="same",activation="relu",input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=2,strides=1))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2,strides=1))
model.add(Dropout(0.2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2,strides=1))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.7))
model.add(Dense(1,activation="sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 32, 32, 128)       1664      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 31, 31, 64)        32832     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 30, 30, 32)        8224      
__________

In [123]:
from keras.callbacks import EarlyStopping
el=EarlyStopping(min_delta=0.001,patience=5)
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
h=model.fit(X,Y,batch_size=256,validation_split=0.2,epochs=100,callbacks=[el])

Train on 14000 samples, validate on 3500 samples
Epoch 1/100
14000/14000 [==============================] - 4s 291us/step - loss: 0.4765 - acc: 0.7552 - val_loss: 0.2471 - val_acc: 0.9329
Epoch 2/100
14000/14000 [==============================] - 3s 195us/step - loss: 0.2760 - acc: 0.8847 - val_loss: 0.2113 - val_acc: 0.9369
Epoch 3/100
14000/14000 [==============================] - 3s 196us/step - loss: 0.2523 - acc: 0.9121 - val_loss: 0.2066 - val_acc: 0.9420
Epoch 4/100
14000/14000 [==============================] - 3s 195us/step - loss: 0.2480 - acc: 0.9087 - val_loss: 0.2013 - val_acc: 0.9460
Epoch 5/100
14000/14000 [==============================] - 3s 195us/step - loss: 0.2196 - acc: 0.9255 - val_loss: 0.1528 - val_acc: 0.9663
Epoch 6/100
14000/14000 [==============================] - 3s 197us/step - loss: 0.2036 - acc: 0.9319 - val_loss: 0.1633 - val_acc: 0.9543
Epoch 7/100
14000/14000 [==============================] - 3s 196us/step - loss: 0.1950 - acc: 0.9387 - val_loss: 0.1

In [124]:
pred=model.predict(X_test)
ids=[]
label=[]
a=0
for i in tqdm(os.listdir(test_path)):
    id=i
    ids.append(id)
    label.append(pred[a])
    a=a+1

label=np.array(label,dtype='float64')
out=pd.DataFrame({'id': ids,'has_cactus':label[:,0]})

out.to_csv('submission.csv',index=False,header=True)

100%|██████████| 4000/4000 [00:00<00:00, 948133.14it/s]
